# Thank you for participating in the AI study!

Dear doctor:

This is the [study consent form](link). Some key points in the consent form:

- Your participation is voluntary
- You may decide to withdraw from this study at any time by closing this webpage

If you have any question regarding the consent form, please feel free to contact the study PI Weina Jin: Phone: *604 603 8530*. Email: *weinaj@sfu.ca*

By clicking the following checkbox, you indicate you have read the consent form and agree to participate in the study. 

In [64]:
import nibabel
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import ipywidgets as widgets
from IPython.core.display import display, HTML
# from IPython.display import IFrame
import warnings
warnings.filterwarnings('ignore')


In [4]:
modality = ['t1', 't1ce', 't2', 'flair']

def load_data(path, bratsID, non_neg=True):
    '''
    Given a path of bratsID, load it with 4 MR modalities, and 4 heatmaps for each modality
    non_neg: get rid of negative value of heatmap
    '''
    
    mri_lst = []
    hm_lst = []
    hm_smooth_lst = []
    for m in modality:
        hm_path = os.path.join(path, bratsID, '{}_heatmap.nii'.format(m.lower())) 
        hm = nibabel.load(hm_path).get_fdata()
        mri_path = os.path.join(path, bratsID, bratsID+'_{}.nii.gz'.format(m.lower())) 
        mri = nibabel.load(mri_path).get_fdata()
        
        hm = hm[::-1,::-1,:]
        if non_neg:
            hm[hm < 0] = 0
        hm_smooth = ndimage.gaussian_filter(hm, sigma=2) 
        
        hm_lst.append(hm)
        mri_lst.append(mri)
        hm_smooth_lst.append(hm_smooth) 
    return mri_lst, hm_lst, hm_smooth_lst

# mri = '/Users/root202/OneDrive - sfu.ca/project/brain/heatmap_eval/heatmap_data/BraTS19_TCIA03_474_1/BraTS19_TCIA03_474_1_t2.nii.gz'
path = '/Users/root202/OneDrive - sfu.ca/project/brain/heatmap_eval/heatmap_data/'


In [5]:
# Code source: https://github.com/mohakpatel/ImageSliceViewer3D

# %matplotlib inline

class ImageSliceViewer3D:
    """ 
    ImageSliceViewer3D is for viewing volumetric image slices in jupyter or
    ipython notebooks. 
    
    User can interactively change the slice plane selection for the image and 
    the slice plane being viewed. 

    Argumentss:
    Volume = 3D input image
    figsize = default(8,8), to set the size of the figure
    cmap = default('plasma'), string for the matplotlib colormap. You can find 
    more matplotlib colormaps on the following link:
    https://matplotlib.org/users/colormaps.html
    
    """
    
    def __init__(self, mri_lst, heatmap_lst, bratsid, alpha=.5, figsize=(10, 10), cmap='gray', heatmap_cmap = 'afmhot'):
        self.mri_lst = mri_lst
        self.heatmap_lst = heatmap_lst
        self.figsize = figsize
        self.cmap = cmap
        self.heatmap_cmap = heatmap_cmap
#         self.v = [np.min(volume), np.max(volume)]
        self.alpha = alpha
        self.hm_on = False
        self.mri_on = True
        self.bratsid = bratsid
        # Call to select slice plane
        widgets.interact(self.view_selection, view=widgets.RadioButtons(
            options=['Axial','Saggital', 'Coronal'], value='Axial', 
            description='MRI view:', disabled=False,
            style={'description_width': 'initial'}))

#         # Call to adjust heatmap alpha
#         widgets.interact(self.heatmap_alpha, alpha=widgets.IntSlider(min=0, max=1, step=0.1, continuous_update=False, 
#             description='Adjust heatmap opaque'))
        
        # Call to turn on/off heatmap
        widgets.interact(self.heatmap_switch, switch = widgets.Checkbox(
                                value=False,
                                description='Show heatmap',
                                disabled=False))
        # Call to turn on/off mri
        widgets.interact(self.mri_switch, switch = widgets.Checkbox(
                                value=True,
                                description='Show MRI',
                                disabled=False))    
    def heatmap_switch(self, switch):
        self.hm_on = switch
        
    def mri_switch(self, switch):
        self.mri_on = switch        
        if self.hm_on==False and self.mri_on == False:
            self.mri_on = True
        
    def view_selection(self, view):
        # Transpose the volume to orient according to the slice plane selection
        orient = {"Saggital":[1,2,0], "Coronal":[2,0,1], "Axial": [0,1,2]}
        for i in range(len(modality)):
            mri= np.transpose(self.mri_lst[i], orient[view])
            hm = np.transpose(self.heatmap_lst[i], orient[view])
            self.mri_lst[i] = mri
            self.heatmap_lst[i] = hm
            maxZ = mri.shape[2] - 1
        
        # Call to view a slice within the selected slice plane
        widgets.interact(self.plot_slice, 
            z=widgets.IntSlider(min=0, max=maxZ, step=1, continuous_update=True, 
            description='Slices:'))
        
#     def heatmap_alpha(self, alpha):
#         widgets.interact(self.plot_slice, 
#             alpha=widgets.IntSlider(min=0, max=1, step=0.1, continuous_update=False, 
#             description='Adjust heatmap opaque'))
        
    def plot_slice(self, z):

        columns = 2
        rows = 2
        # Plot slice for the given plane and slice
        self.fig = plt.figure(figsize=self.figsize)
        for i in range(1, columns*rows +1):
            mri = self.mri_lst[i-1]
            hm = self.heatmap_lst[i-1]
            subplot = self.fig.add_subplot(rows, columns, i)
            if self.hm_on==False and self.mri_on == False:
                self.mri_on = True
            if self.mri_on:
                subplot.imshow(mri[:,:,z], cmap=plt.get_cmap(self.cmap), 
                    vmin=np.min(mri), vmax=np.max(mri))
            if self.hm_on:
    #             hm_mask = np.ma.masked_array(mri*hm > 0, hm)
                subplot.imshow(hm[:,:,z], cmap=plt.get_cmap(self.heatmap_cmap),
                       alpha=self.alpha, vmin=np.min(hm), vmax=np.max(hm))
    #             plt.imshow(hm_mask[:,:,z], cmap='gray', alpha=0.5)
            subplot.set_title(modality[i-1].upper()+' '+self.bratsid)

In [58]:
bratsID_lst = [\
               'BraTS19_TCIA03_474_1',
               'BraTS19_TCIA01_412_1' 
]

# survey_lst  = [\
# '<script>(function(t,e,s,n){var o,a,c;t.SMCX=t.SMCX||[],e.getElementById(n)||(o=e.getElementsByTagName(s),a=o[o.length-1],c=e.createElement(s),c.type="text/javascript",c.async=!0,c.id=n,c.src="https://widget.surveymonkey.com/collect/website/js/tRaiETqnLgj758hTBazgd5IivfPEhj_2FUp_2BRFniVOKyrFsYUvE_2FVb6VXODPcH63Mv.js",a.parentNode.insertBefore(c,a))})(window,document,"script","smcx-sdk");</script>',
# '<script>(function(t,e,s,n){var o,a,c;t.SMCX=t.SMCX||[],e.getElementById(n)||(o=e.getElementsByTagName(s),a=o[o.length-1],c=e.createElement(s),c.type="text/javascript",c.async=!0,c.id=n,c.src="https://widget.surveymonkey.com/collect/website/js/tRaiETqnLgj758hTBazgd5IivfPEhj_2FUp_2BRFniVOKyprw0DnlPrX6JtEzxFh8X8u.js",a.parentNode.insertBefore(c,a))})(window,document,"script","smcx-sdk");</script>'
#               ]

survey_lst = [\
              '<iframe width="640px" height= "480px" src= "https://forms.office.com/Pages/ResponsePage.aspx?id=fmfoBInJuUeGGdg9Wl9sZ12EXzfHI9lKlQYyBLdG2lpUQzFUSFJHWlo5T0VQQVJKT0JMV1dOT1lYVy4u&embed=true" frameborder= "0" marginwidth= "0" marginheight= "0" style= "border: none; max-width:100%; max-height:100vh" allowfullscreen webkitallowfullscreen mozallowfullscreen msallowfullscreen> </iframe>',
              '<iframe width="640px" height= "480px" src= "https://forms.office.com/Pages/ResponsePage.aspx?id=fmfoBInJuUeGGdg9Wl9sZ12EXzfHI9lKlQYyBLdG2lpUQzFUSFJHWlo5T0VQQVJKT0JMV1dOT1lYVy4u&embed=true" frameborder= "0" marginwidth= "0" marginheight= "0" style= "border: none; max-width:100%; max-height:100vh" allowfullscreen webkitallowfullscreen mozallowfullscreen msallowfullscreen> </iframe>'
]
assert len(bratsID_lst) == len(survey_lst)
def show_mri(b):
    if b:
        for mri_idx in range(len(bratsID_lst)):
            mri_lst, hm_lst, hm_smooth_lst = load_data(path, bratsID_lst[mri_idx])
            progress = widgets.HTML(value="<h1>MRI {} / {} </h1>".format(mri_idx+1, len(bratsID_lst)))
            display(progress)

            ImageSliceViewer3D(mri_lst, hm_smooth_lst, bratsID_lst[mri_idx])
            display(HTML(survey_lst[mri_idx]))#, width = 500, height =500)


In [59]:
consent_check = widgets.Checkbox(
    value=False,
    description='I give consent to the study',
    disabled=False
)

caption = widgets.HTML(value="<h2>Thanks for your participation in the study! Let's start reading MRIs!</h2>")

tutorial = widgets.HTML(
    value="<h3>In this study you are invited to view several MRIs and give your judgment. </h3>\
    <h4>Here is a tutorial on how to read MRI</h4>")

gif = open("../gif_placeholder.gif", "rb").read()
tutorial_gif = widgets.Image(
                    value=gif,
                    format='gif',
                    width=300,
                    height=400,
                )


def consent(b):
    if b:
        display(caption,tutorial, tutorial_gif)
        show_mri(True)
        consent_check.close()
        

widgets.interact(consent, b=consent_check)
output = widgets.Output()


interactive(children=(Checkbox(value=False, description='I give consent to the study'), Output()), _dom_classe…